## ETL for MN DNR LIDAR/las data 


#### https://resources.gisdata.mn.gov/pub/data/elevation/lidar/

## Import modules & set workspace

In [ ]:
from ftplib import FTP # For downloading/extracting zip files
import arcpy   # For converting las to TIN & DEM

# Set workspace to ArcPro folder
arcpy.env.workspace = r'C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\Lab2_take2.aprx'

## Collect & prep data

In [ ]:
# Connect to ftp server

ftp = FTP('ftp.lmic.state.mn.us')  # Connect to host with default port
ftp.login()                     # Use anonymous credentials
'230 Login successful.'
ftp.cwd('/pub/data/elevation/lidar/projects/lakeshady/laz/')  # Change into "lidar" directory
ftp.retrlines('LIST')           # List directory contents


In [ ]:
# Download as binary 
with open('4342-28-47.laz', 'wb') as fp:    
    ftp.retrbinary('RETR 4342-28-47.laz', fp.write)  # FTP's RETR command downloads file

#ftp.quit()  # Can quit when transfers are complete

In [ ]:
# .Laz must be converted to a .las to be opened in Pro 
arcpy.conversion.ConvertLas(r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\4342-28-47.laz", r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\las_files", "SAME_AS_INPUT", '', "NO_COMPRESSION", None, r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\las_files\4342-28-47.lasd")

## Convert .las to TIN

### Tool doc for TIN 
#### arcpy.3d.LasDatasetToTin(in_las_dataset, out_tin, {thinning_type}, {thinning_method}, {thinning_value}, {max_nodes}, {z_factor}, {clip_to_extent})

In [ ]:
# Conversion to TIN

arcpy.ddd.LasDatasetToTin("4342-28-47.lasd", "lakeShady_tin","RANDOM", "PERCENT", 20)

### Tool doc for DEM
#### arcpy.conversion.LasDatasetToRaster(in_las_dataset, out_raster, {value_field}, {interpolation_type}, {data_type}, {sampling_type}, {sampling_value}, {z_factor})

In [ ]:
# Conversion to DEM

arcpy.conversion.LasDatasetToRaster("4342-28-47.lasd", "lakeShady_rast", "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", 10, 1)

## Create a map layout & export as PDF

In [ ]:
# Create layout - Is this possible in code?

### Export as a pdf

In [ ]:
import arcpy
# Define the map path
aprx = arcpy.mp.ArcGISProject(r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\5572_Lab2.aprx")

# Define & reference TIN & DEM map layouts
dem = aprx.listLayouts("DEM")[0]  
tin = aprx.listLayouts("TIN")[0]

# Export layouts to PDF
dem.exportToPDF(r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\pdfs\DEM.pdf", resolution = 300)
tin.exportToPDF(r"C:\Users\mmMary\Documents\GIS_Classes\GIS_5572\Labs\Lab2\pdfs\TIN.pdf", resolution = 300)